<a href="https://colab.research.google.com/github/melihagokcekose/GenetikAlgoritma/blob/main/Bitirme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [2]:
import random
import string
import pandas as pd
import random
import string
import re
from datasets import load_dataset

In [3]:
#dataset = load_dataset("Mursel/Turkish-wikipedia-10k")
#dataset = load_dataset("TFLai/Turkish-Dialog-Dataset") #minik dataset
dataset2 = load_dataset("erenfazlioglu/turkishwikipedia2023") #558 mb
#dataset = load_dataset("umarigan/turkish_wikipedia") #630 mb
# #texts = dataset["train"]["content"][:10]  # Use a subset for demonstration
#texts = dataset2["train"]["text"][:10]
dataset2["train"]["text"][:1]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

['Cengiz Han (doğum adıyla Temuçin, – 18 Ağustos 1227), Moğol İmparatorluğu\'nun kurucusu ve ilk Kağanı olan Moğol komutan ve hükümdardır. Hükümdarlığı döneminde gerçekleştirdiği hiçbir savaşı kaybetmeyen Cengiz Han, dünya tarihinin en büyük askeri liderlerinden birisi olarak kabul edilmektedir. 13. yüzyılın başında Orta Asya\'daki tüm göçebe bozkır kavimlerini birleştirip bir ulus hâline getirerek Moğol siyasi kimliği çatısı altında toplamıştır. Cengiz Han, hükümdarlığı döneminde, 1206-1227 arasında, Kuzey Çin\'deki Batı Xia ve Jin Hanedanı; Türkistan\'daki Kara Hıtay, Maveraünnehir; Harezm, Horasan ve İran\'daki Harezmşahlar, Kafkasya\'daki Gürcüler, Deşt-i Kıpçak\'taki Rus Knezlikleri, Kıpçaklar ile İdil Bulgarları üzerine seferler yaptı ve imparatorluğu döneminde gerçekleştirdiği hiçbir savaşı kaybetmedi. Bunların sonucunda Pasifik Okyanusu\'ndan Hazar Denizi\'ne ve Karadeniz\'in kuzeyine kadar uzanan bir imparatorluk kurdu. 1

In [4]:
train_dataset = dataset2["train"]["text"]
len(train_dataset)

555338

In [5]:
!pip install nltk

In [6]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from datasets import load_dataset

# Cümle ayırıcı
nltk.download('punkt')

# Cümleleri ayır ve DataFrame oluştur
sentences_list = []
for text in train_dataset:
    sentences = sent_tokenize(text)
    sentences_list.extend(sentences)

df = pd.DataFrame({"Sentences": sentences_list})

# DataFrame'i yazdır (Temizlenmemiş cümlelerden oluşuyor)
#print(df)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
df.head(5)

,Sentences
0,"Cengiz Han (doğum adıyla Temuçin, – 18 Ağus..."
1,Hükümdarlığı döneminde gerçekleştirdiği...
2,13. yüzyılın başında Orta Asya'daki tüm gö...
3,"Cengiz Han, hükümdarlığı döneminde, 1206-1..."
4,Bunların sonucunda Pasifik Okyanusu'ndan Hazar...


In [8]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-ZğüşöçĞÜŞÖÇİı\s]', '', text)
    text = text.lower()
    return text

# Boş bir DataFrame oluştur
#dfc = pd.DataFrame(columns=["cleaned_text"])

# Her bir metni temizle ve DataFrame'e ekle
#for text in texts:
#    cleaned_text = clean_text(text)
#    dfc = dfc.append({"cleaned_text": cleaned_text}, ignore_index=True)

# Temizlenmiş DataFrame
#print(dfc)


In [9]:
df['cleaned'] = df['Sentences'].apply(clean_text)

In [10]:
#Augmentation İşlemleri

def permute_letter(word):
    permute_list = [word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(len(word)-1)]
    return permute_list

def delete_letter(word):
    delete_list = [word[:i] + word[i+1:] for i in range(len(word))]
    return delete_list

def insert_letter(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_list = [word[:i] + letter + word[i:] for i in range(len(word)+1) for letter in letters]
    return insert_list

def replace_letter(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_list = [word[:i] + letter + word[i+1:] for i in range(len(word)) for letter in letters]
    return replace_list

def noising_strategy(word):
    operations = ["permute", "delete", "insert", "replace"]
    selected_operation = random.choices(operations, weights=[0.1, 0.1, 0.1, 0.1])[0]

    if len(word) < 3:
        return [word]

    if selected_operation == "permute":
        return permute_letter(word)
    elif selected_operation == "delete":
        return delete_letter(word)
    elif selected_operation == "insert":
        return insert_letter(word)
    elif selected_operation == "replace":
        return replace_letter(word)

def data_augmentation(sentence, rate=0.1):
    words = sentence.split()
    augmented_words = []

    for word in words:
        if random.random() <= rate:
            augmented_word = random.choice(noising_strategy(word))
        else:
            augmented_word = word

        augmented_words.append(augmented_word)

    return ' '.join(augmented_words)


In [11]:
df["random corrupted"] = df["cleaned"].apply(lambda x: data_augmentation(x, rate=0.1))

# Augmentation sonucunu yazdır
print("\nRandom corrupted:")
df.head(5)


Random corrupted:


,Sentences,cleaned,random corrupted
0,"Cengiz Han (doğum adıyla Temuçin, – 18 Ağus...",cengiz han dogum adıyla temucin agustos mog...,cengiz han dogum adıyla temucin agustos mogol ...
1,Hükümdarlığı döneminde gerçekleştirdiği...,hukumdarlıgı doneminde gerceklestirdigi hicbir...,hukumdarlıgı doneminde gerceklestirdigi hicbir...
2,13. yüzyılın başında Orta Asya'daki tüm gö...,yuzyılın basında orta asyadaki tum gocebe boz...,yuzyılın basında orta asyadaki tum gocebe bozk...
3,"Cengiz Han, hükümdarlığı döneminde, 1206-1...",cengiz han hukumdarlıgı doneminde arasında ku...,cengiz han hukumdarlıgı doneminde arasında kuz...
4,Bunların sonucunda Pasifik Okyanusu'ndan Hazar...,bunların sonucunda pasifik okyanusundan hazar ...,bunların sonucunda lasifik okyanusundan hazar ...


In [12]:
df.to_csv('train_set.csv')

In [13]:
!pip install numpy